[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huhlim/alphafold-multistate/blob/main/AlphaFold_multistate.ipynb)

#Multi-state modeling of the GPCR and kinase using AlphaFold2
It is a modified version of multi-state modeling protocol for the GPCR and kinase using the [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) interface. The original protocol utlizes [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2) and slightly modified to model multiple states of the GPCR and kinase. Modeling is guided by templates selected from activation state-annotated GPCR databases or conformational state-annotated kinase databases. The state annotations were taken from [GPCRdb](https://gpcrdb.org/structure/) for the GPCR and [KinCoRe](http://dunbrack.fccc.edu/kincore/home) for the kinase. To effectively use structural template-based features, MSA-based features are further modified. For details, please check [our manuscript](https://www.biorxiv.org/content/10.1101/2021.11.26.470086v2) and [our GitHub repository](https://github.com/huhlim/alphafold-multistate).



In [ ]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`
from google.colab import files
import os.path
import re
import hashlib

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

query_sequence = 'MEDFNMESDSFEDFWKGEDLSNYSYSSTLPPFLLDAAPCEPESLEINKYFVVIIYALVFL LSLLGNSLVMLVILYSRVGRSVTDVYLLNLALADLLFALTLPIWAASKVNGWIFGTFLCK VVSLLKEVNFYSGILLLACISVDRYLAIVHATRTLTQKRYLVKFICLSIWGLSLLLALPV LLFRRTVYSSNVSPACYEDMGNNTANWRMLLRILPQSFGFIVPLLIMLFCYGFTLRTLFK AHMGQKHRAMRVIFAVVLIFLLCWLPYNLVLLADTLMRTQVIQETCERRNHIDRALDATE ILGILHSCLNPLIYAFIGQKFRHGLLKILAIHGLISKDSLPKDSRPSFVGSSSGHTSTTL' #@param {type:"string"}

# remove whitespaces
query_sequence = "".join(query_sequence.split())

jobname = 'CXCR2_HUMAN' #@param {type:"string"}
# remove whitespaces
jobname = "".join(jobname.split())
jobname = re.sub(r'\W+', '', jobname)
jobname = add_hash(jobname, query_sequence)

protein_family = "GPCR" #@param ["GPCR", "Kinase"]
gpcr_state = "Active" #@param ["None", "Active", "Inactive"]
kinase_state = "None" #@param ["None", "DFGin_All", "DFGin_ABAminus", "DFGin_BLAminus", "DFGin_BLAplus", "DFGin_BLBminus", "DFGin_BLBplus", "DFGin_BLBtrans", "DFGin_Other", "DFGinter_All", "DFGinter_BABtrans", "DFGinter_Other", "DFGout_All", "DFGout_BBAminus", "DFGout_Other", "Other_All"]
if protein_family == "GPCR":
  if gpcr_state == "None":
    raise ValueError("Please select a gpcr_state")
  kinase_state = "None"
elif protein_family == "Kinase":
  if kinase_state == "None":
    raise ValueError("Please select a kinase_state")
  gpcr_state = "None"

with open(f"{jobname}.csv", "w") as text_file:
    text_file.write(f"id,sequence\n{jobname},{query_sequence}")

queries_path=f"{jobname}.csv"


msa_mode = "MMseqs2 (UniRef+Environmental)"
model_type = "auto"
num_models = 5
num_recycles = 3
use_amber = False
use_templates = True

#@markdown ---
#@markdown ### Experimental options
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form.

# decide which a3m to use
if msa_mode.startswith("MMseqs2"):
  a3m_file = f"{jobname}.a3m"
else:
  a3m_file = f"{jobname}.single_sequence.a3m"
  with open(a3m_file, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

if save_to_google_drive:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("You are logged into Google Drive and are good to go!")

In [ ]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  echo "installing colabfold..."
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    echo "installing conda..."
    wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
    bash Mambaforge-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Mambaforge-Linux-x86_64.sh
    mamba config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" gdown 2>&1 1>/dev/null
  touch HH_READY
fi

if [ ! -e colabfold_runner.py ]; then
  wget -q https://raw.githubusercontent.com/huhlim/alphafold-multistate/main/structure_prediction/colabfold_runner.py
fi

In [ ]:
#@title Download conformational state-annotated databases
import pathlib
import subprocess as sp

if protein_family == "GPCR":
    conformational_state = gpcr_state
    db_home = pathlib.Path("gpcr100")
    mmcif_home = db_home / "mmcif"
    db_home.mkdir(exist_ok=True)
    mmcif_home.mkdir(exist_ok=True)
elif protein_family == "Kinase":
    conformational_state = kinase_state
    db_home = pathlib.Path("kinase100")
    db_home.mkdir(exist_ok=True)

if protein_family == "GPCR":
    hh_tgz = f"GPCR100.{conformational_state}.tgz"
    cif_tgz = f"cif.{conformational_state}.tgz"
    if not os.path.exists(hh_tgz):
        sp.call(["wget", "-q", "-c", f"https://zenodo.org/records/7860658/files/{hh_tgz}"])
        sp.call(["tar", "xzf", f"GPCR100.{conformational_state}.tgz", "-C", db_home])
    if not os.path.exists(cif_tgz):
        sp.call(["wget", "-q", "-c", f"https://zenodo.org/records/7860658/files/{cif_tgz}"])
        sp.call(["tar", "xzf", f"cif.{conformational_state}.tgz", "-C", db_home])
        cif_fn_s = list((db_home/f"cif.{conformational_state}").glob("*.cif"))
        sp.call(["mv"] + cif_fn_s + [mmcif_home])

elif protein_family == "Kinase":
    hh_tgz = f"kinase100.{conformational_state}.tgz"
    cif_tgz = f"cif.kinase.tgz"
    if not os.path.exists(hh_tgz):
        sp.call(["wget", "-q", "-c", f"https://zenodo.org/records/7860658/files/{hh_tgz}"])
        sp.call(["tar", "xzf", f"kinase100.{conformational_state}.tgz", "-C", db_home])
    if not os.path.exists(cif_tgz):
        sp.call(["wget", "-q", "-c", f"https://zenodo.org/records/7860658/files/{cif_tgz}"])
        sp.call(["tar", "xzf", f"cif.kinase.tgz", "-C", db_home])



In [ ]:
#@title Prepare Prediction

import sys

from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, set_model_type
K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
if "1" in K80_chk:
  print("WARNING: found GPU Tesla K80: limited to total length < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt
from colabfold.plot import plot_msa_v2

display_images = True

def prediction_callback(protein_obj, length,
                        prediction_result, input_features, mode):
  model_name, relaxed = mode
  if not relaxed:
    if display_images:
      fig = plot_protein(protein_obj, Ls=length, dpi=150)
      plt.show()
      plt.close()

result_dir = jobname
if 'logging_setup' not in globals():
    setup_logging(Path(os.path.join(jobname,"log.txt")))
    logging_setup = True
queries, is_complex = get_queries(queries_path)
model_type = set_model_type(is_complex, model_type)
download_alphafold_params(model_type, Path("."))

In [ ]:
#@title Run Prediction

import colabfold_runner

results = colabfold_runner.run(
    queries=queries,
    result_dir=result_dir,
    protein_family=protein_family,
    conformational_state=conformational_state,
    use_templates=use_templates,
    custom_template_path=None,
    use_amber=use_amber,
    msa_mode=msa_mode,
    model_type=model_type,
    num_models=5,
    num_recycles=num_recycles,
    model_order=[1, 2],
    is_complex=is_complex,
    data_dir=Path("."),
    keep_existing_results=False,
    rank_by="auto",
    stop_at_score=float(100),
    prediction_callback=prediction_callback,
)

results_zip = f"{jobname}.result.zip"
os.system(f"zip -r {results_zip} {jobname}")

In [ ]:
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend
from colabfold.colabfold import pymol_color_list, alphabet_list
rank_num = 1 #@param ["1", "2"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

tag = results["rank"][0][rank_num - 1]
jobname_prefix = ".custom" if msa_mode == "custom" else ""
pdb_filename = f"{jobname}/{jobname}{jobname_prefix}_unrelaxed_{tag}.pdb"
pdb_file = glob.glob(pdb_filename)

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
#@title Plots {run: "auto"}
from IPython.display import display, HTML
import base64
from html import escape

# see: https://stackoverflow.com/a/53688522
def image_to_data_url(filename):
  ext = filename.split('.')[-1]
  prefix = f'data:image/{ext};base64,'
  with open(filename, 'rb') as f:
    img = f.read()
  return prefix + base64.b64encode(img).decode('utf-8')

pae = image_to_data_url(os.path.join(jobname,f"{jobname}{jobname_prefix}_pae.png"))
cov = image_to_data_url(os.path.join(jobname,f"{jobname}{jobname_prefix}_coverage.png"))
plddt = image_to_data_url(os.path.join(jobname,f"{jobname}{jobname_prefix}_plddt.png"))
display(HTML(f"""
<style>
  img {{
    float:left;
  }}
  .full {{
    max-width:100%;
  }}
  .half {{
    max-width:50%;
  }}
  @media (max-width:640px) {{
    .half {{
      max-width:100%;
    }}
  }}
</style>
<div style="max-width:90%; padding:2em;">
  <h1>Plots for {escape(jobname)}</h1>
  <img src="{pae}" class="full" />
  <!--img src="{cov}" class="half" /-->
  <img src="{plddt}" class="half" />
</div>
"""))


In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).


if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

# Instructions <a name="Instructions"></a>
This Colab notebook is mostly based on [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb). We (Lim Heo and Michael Feig) slightly modified the notebook for multi-state modeling of the GPCR.


**Quick start**
1. Paste your protein sequence(s) in the input field.
2. Specify the **activation state** for modeling.
3. Press "Runtime" -> "Run all".

**Result zip file contents**
1. PDB formatted structures sorted by avg. pIDDT and complexes are sorted by pTMscore. (unrelaxed and relaxed if `use_amber` is enabled).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. BibTeX file with citations for all used tools and databases.

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Additionally, if the `save_to_google_drive` option was selected, the `jobname.result.zip` will be uploaded to your Google Drive.

**Troubleshooting**
* Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* If a prediction failed at the step of "Run Prediction", it is usually because of incompatibilities between dependent libraries including dm-haiku and jax, which are actively developed. If you encounter any errors, please let us know at huhlim@gmail.com or https://github.com/huhlim/alphafold-multistate/issues.
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* Computing resources: MMseqs2 API can handle ~20-50k requests per day.
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or Mgnify.

**Description of the plots**
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.


**Acknowledgments**
- [DeepMind](https://github.com/deepmind/alphafold) for the development and sharing the excellent protein structure prediction method, AlphaFold2.
- [ColabFold](https://github.com/sokrypton/ColabFold) for providing the valuable interface for running AlphaFold.
- [KOBIC](https://kobic.re.kr) and [Söding Lab](https://www.mpinat.mpg.de/soeding) for providing the computational resources for the MMseqs2 MSA server.


